
# Retrieval-Augmented Generation (RAG) — Free / No API Key Version
This notebook demonstrates a full **Retrieval-Augmented Generation (RAG)** pipeline using only free, local models.

You’ll:
1. Create a small knowledge base  
2. Chunk and embed text using Hugging Face (no API key)  
3. Store and retrieve chunks with Chroma  
4. Generate grounded answers with FLAN‑T5 (local model)

**Screenshots:**  
- Library install success  
- Loaded docs / created chunks  
- Vector store ready  
- Final Q/A answer  


## 1️⃣ Install libraries

In [ ]:

!pip install -q langchain langchain-community chromadb sentence-transformers transformers accelerate pypdf
print("✅ All libraries installed")


✅ All libraries installed


## 2️⃣ Load and split documents

In [ ]:

from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_DIR = Path("data_rag_demo")
loader = DirectoryLoader(str(DATA_DIR), glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = splitter.split_documents(docs)

print(f"📄 Loaded {len(docs)} docs → 🔪 Created {len(splits)} chunks")


📄 Loaded 3 docs → 🔪 Created 3 chunks


## 3️⃣ Build local Chroma vector store (no API key)

In [ ]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

PERSIST_DIR = "chroma_db_free"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(splits, embedding=embeddings, persist_directory=PERSIST_DIR)
vectordb.persist()
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

print("✅ Local Chroma vector store ready at:", PERSIST_DIR)


✅ Local Chroma vector store ready at: chroma_db_free


/tmp/ipython-input-2581740092.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
import zipfile, os, shutil

zip_name = "RAG_file.zip"   # must match the name shown in the left panel

# Unzip the file
with zipfile.ZipFile(zip_name, "r") as zip_ref:
    zip_ref.extractall(".")

print("📂 After unzip, I see:", os.listdir("."))

# Move the data folder to current working directory if needed
if os.path.exists("RAG_file/data_rag_demo"):
    shutil.move("RAG_file/data_rag_demo", ".")
    print("✅ Moved data_rag_demo into working directory")

print("📄 data_rag_demo contents:", os.listdir("data_rag_demo"))


📂 After unzip, I see: ['.config', 'chroma_db_free', 'RAG_Free_No_API.ipynb', 'RAG_file.zip', 'data_rag_demo', 'sample_data']
📄 data_rag_demo contents: ['rag_intro.txt', 'best_practices.txt', 'vector_stores.txt']


In [ ]:
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_DIR = Path("data_rag_demo")

loader = DirectoryLoader(str(DATA_DIR), glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = splitter.split_documents(docs)

print(f"📄 Loaded {len(docs)} docs → 🔪 Created {len(splits)} chunks")


📄 Loaded 3 docs → 🔪 Created 3 chunks


In [ ]:
print("Number of docs:", len(docs))
print("Number of chunks:", len(splits))

for i, d in enumerate(docs):
    print(f"[DOC {i}] source={d.metadata.get('source')} preview={d.page_content[:120]!r}")


Number of docs: 3
Number of chunks: 3
[DOC 0] source=data_rag_demo/rag_intro.txt preview='Retrieval-Augmented Generation (RAG) retrieves relevant context from a knowledge base and passes it to a language model '
[DOC 1] source=data_rag_demo/best_practices.txt preview='Good RAG design uses chunk overlaps, stores metadata for traceability, and evaluates answers for faithfulness to the ret'
[DOC 2] source=data_rag_demo/vector_stores.txt preview='Vector stores like Chroma store text embeddings so we can search by semantic meaning, not just exact keywords.'


## 4️⃣ Load free local model (FLAN‑T5)

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("✅ Model loaded:", model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Model loaded: google/flan-t5-base


## 5️⃣ Ask questions (RAG pipeline)

In [ ]:
import os, shutil
os.makedirs("data_rag_demo", exist_ok=True)
for f in ["rag_intro.txt", "vector_stores.txt", "best_practices.txt"]:
    if os.path.exists(f):
        shutil.move(f, "data_rag_demo/")
print("✅ Folder ready with files:", os.listdir("data_rag_demo"))


✅ Folder ready with files: ['rag_intro.txt', 'best_practices.txt', 'vector_stores.txt']


In [ ]:

def generate_answer_from_context(question, context_text):
    prompt = (
        "Use ONLY the context below to answer the question. "
        "If the answer is not in the context, say you don't know.\n\n"
        f"Context:\n{context_text}\n\nQuestion: {question}\nAnswer:"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256, temperature=0.0)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def rag_query(question):
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([d.page_content for d in docs])
    answer = generate_answer_from_context(question, context)
    print("Q:", question)
    print("\nContext used:\n", context[:500], "...\n")
    print("A:", answer)

rag_query("What is RAG and why do we use a vector store?")


/tmp/ipython-input-708094551.py:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Q: What is RAG and why do we use a vector store?

Context used:
 Good RAG design uses chunk overlaps, stores metadata for traceability, and evaluates answers for faithfulness to the retrieved context.
Vector stores like Chroma store text embeddings so we can search by semantic meaning, not just exact keywords.
Retrieval-Augmented Generation (RAG) retrieves relevant context from a knowledge base and passes it to a language model for grounded answers. ...

A: retrievs relevant context from a knowledge base and passes it to a language model for grounded answers


In [ ]:
def generate_answer_from_context(question, context_text):
    prompt = (
        "Use ONLY the context below to answer the question. "
        "If the answer is not in the context, say you don't know.\n\n"
        f"Context:\n{context_text}\n\n"
        f"Question: {question}\nAnswer:"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            temperature=0.0,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def rag_query(question):
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([d.page_content for d in docs])

    answer = generate_answer_from_context(question, context)

    print("Q:", question)
    print("\nContext used:\n", context[:600], "...\n")
    print("A:", answer)

    return answer

rag_query("What is RAG and why do we use a vector store?")


Q: What is RAG and why do we use a vector store?

Context used:
 Good RAG design uses chunk overlaps, stores metadata for traceability, and evaluates answers for faithfulness to the retrieved context.
Vector stores like Chroma store text embeddings so we can search by semantic meaning, not just exact keywords.
Retrieval-Augmented Generation (RAG) retrieves relevant context from a knowledge base and passes it to a language model for grounded answers. ...

A: retrievs relevant context from a knowledge base and passes it to a language model for grounded answers


'retrievs relevant context from a knowledge base and passes it to a language model for grounded answers'

In [1]:
from google.colab import files
import json, requests, re, os

# This block grabs the current notebook name from the Colab UI:
notebook_name = "RAG_Assignment.ipynb"  # <-- if you already named it something, put that name here

print("✅ Assuming current notebook is:", notebook_name)


✅ Assuming current notebook is: RAG_Assignment.ipynb


In [6]:
import nbformat
import json

bad_path = "RAG_Assignment.ipynb"          # <-- your current notebook
clean_path = "RAG_Assignment_CLEAN.ipynb"  # <-- we'll create this cleaned file

# 1. Load the notebook
nb = nbformat.read(bad_path, as_version=4)

# 2. Remove bad widget metadata from notebook-level metadata
if "widgets" in nb.metadata:
    # GitHub expects a "state" field inside widgets metadata
    widgets_meta = nb.metadata["widgets"]
    if not isinstance(widgets_meta, dict) or "state" not in widgets_meta:
        # if it's malformed, just delete it
        del nb.metadata["widgets"]

# 3. Remove widget metadata from each cell too (some cells carry their own junk)
for cell in nb.cells:
    if "metadata" in cell:
        # Colab sometimes leaves "id", "colab", "outputId" etc., those are safe.
        # We're only scrubbing broken widget metadata.
        if "widgets" in cell.get("metadata", {}):
            w = cell["metadata"]["widgets"]
            if not isinstance(w, dict) or "state" not in w:
                del cell["metadata"]["widgets"]

# 4. Save cleaned notebook
nbformat.write(nb, clean_path)

print("✅ Cleaned notebook written to", clean_path)


FileNotFoundError: [Errno 2] No such file or directory: 'RAG_Assignment.ipynb'